# Seperability Insight into OPT models
Tests to see if it is possible to remove coding ability from Meta OPT model for different scales.
Current methods are:
- look at activation frequency of MLP mid layers
- Look at 'crossover threshold' of Attention heads

In [5]:
try: # if in google colab, download necessary python files
  import google.colab 
  ! git clone https://github.com/pesvut/opt-tools.git && mv ./opt-tools/src/*.py .
except ModuleNotFoundError:
  pass
! pip3 install -qqr requirements.txt 

In [6]:
import torch
import numpy as np
import pandas as pd
import einops
import matplotlib.pyplot as plt
import wandb

from model import Model
from texts import prepare
from activations import prune_and_evaluate, evaluate_all, init_data_dict

# df.append has FutureWarning, here is the alternative
def append( df: pd.DataFrame, data: dict ):
    wandb.log(data)
    data = pd.DataFrame({ k:[v] for k,v in data.items() })
    return pd.concat([ df, data ], ignore_index=True )

In [7]:
# Configure initial model and tests
model_size, token_limit  = "125m", 1000
run_pre_test             = True
pre_removals = []

# Removals parameters
ff_frac,   ff_eps   = 0.02, 0.001
attn_frac           = 0.01
cripple, focus      = "code", "pile"

In [8]:
# Prepare data logging
wandb.init(project="seperability-code-pile")
c = wandb.config
c.update({
    "model_size"  : model_size,
    "token_limit" : token_limit,
    "run_pre_test": run_pre_test,
    "ff_frac"  : ff_frac,
    "ff_eps"   : ff_eps,
    "attn_frac": attn_frac,
    "cripple": cripple,
    "focus"  : focus,
})
df = pd.DataFrame()

# Load model and show details about model
opt = Model( c.model_size, limit=c.token_limit )

# Pre-pruning of model
opt.delete_ff_keys_from_files(pre_removals)

- Loaded OPT-125m
 - Registered 12 OPT Attention Layers


In [9]:
# Evaluate model before removal of any neurons
if c.run_pre_test:
    data = init_data_dict()
    data.update( evaluate_all( opt, 1e5 ) )
    df = append( df, data )

Using custom data configuration EleutherAI--the_pile_deduplicated-40e90bd48c49557b
Acc: 72.93|44.97 (Skip: 60.72|34.49): : 10096it [00:08, 1162.16it/s]                          
Using custom data configuration codeparrot--codeparrot-clean-valid-826c6fd8b27e5523
Acc: 79.51|60.27 (Skip: 63.43|42.91): : 10209it [00:07, 1282.54it/s]                          


In [10]:
# First do some pruning of the feed forward layers
for i in range(3):
    data = prune_and_evaluate( opt, c.ff_frac, c.attn_frac, c.ff_eps, cripple=c.cripple, focus=c.focus )
    df = append( df, data )
print(df.T)

Using custom data configuration EleutherAI--the_pile_deduplicated-40e90bd48c49557b
100311it [01:35, 1050.61it/s]                             
Using custom data configuration codeparrot--codeparrot-clean-valid-826c6fd8b27e5523
  2%|▏         | 2000/100000.0 [00:02<01:47, 909.67it/s]

In [ ]:
# Next do some pruning of the feed forward layers + attention layers
for i in range(3):
    data = prune_and_evaluate( opt, c.ff_frac, c.attn_frac, c.ff_eps, cripple=c.cripple, focus=c.focus )
    df = append( df, data )
print(df.T)